## 4. Comprehensions and Generators

### 31 Be Defensive When Iterating Over Arguments

In [1]:
import logging

In [2]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [3]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [4]:
path = 'my_numbers.txt'
with open(path, 'w') as f:
    for i in (15, 35, 80):
        f.write('%d\n' % i)

In [5]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [6]:
it = read_visits('my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


In [7]:
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it))  # Already exhausted

[15, 35, 80]
[]


In [8]:
def normalize_copy(numbers):
    numbers_copy = list(numbers)  # Copy the iterator
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result


In [9]:
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [10]:
def normalize_func(get_iter):
    total = sum(get_iter())   # New iterator
    result = []
    for value in get_iter():  # New iterator
        percent = 100 * value / total
        result.append(percent)
    return result

In [11]:
path = 'my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path))
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [12]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path

    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [13]:
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [14]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers:  # An iterator -- bad!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [15]:
visits = [15, 35, 80]
normalize_defensive(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [16]:
it = iter(visits)
try:
    normalize_defensive(it)
except TypeError:
    pass
else:
    assert False

In [17]:
from collections.abc import Iterator 

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator):  # Another way to check
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [18]:
visits = [15, 35, 80]
normalize_defensive(visits)

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [19]:
it = iter(visits)
try:
    normalize_defensive(it)
except TypeError:
    pass
else:
    assert False

In [20]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

In [21]:
try:
    visits = [15, 35, 80]
    it = iter(visits)
    normalize_defensive(it)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-21-6fa8007bd17c>", line 4, in <module>
    normalize_defensive(it)
  File "<ipython-input-17-528b25b2a0f1>", line 5, in normalize_defensive
    raise TypeError('Must supply a container')
TypeError: Must supply a container


> - 입력 인자를 여러 번 이터레이션하는 함수나 메서드를 조심하라. 입력받은 인자가 이터레이터면 함수가 이상하게 작동하거나 결과가 없을 수 있다.
> - 파이썬의 이터레이터 프로토콜은 컨테이너와 이터레이터가 `iter`, `next` 내장 함수나 `for` 루프 등의 관련 식과 상호작용하는 절차를 정의한다.
> - `__iter__` 메서드를 제너레이터로 정의하면 쉽게 이터러블 컨테이너 타입을 정의할 수 있다.
> - 어떤 값이 (컨테이너가 아닌) 이터레이터인지 감지하려면, 이 값을 `iter` 내장 함수에 넘겨서 반환되는 값이 원래 값과 같은지 확인하면 된다. 다른 방법으로 `collections.abc.Iterator` 클래스를 `isinstance`와 함께 사용할 수도 있다.